# Glacier Ice Thickness Change Analysis
## Sikkim, Bhutan, and Arunachal Pradesh (High Mountain Asia)

This notebook analyzes glacier surface elevation changes using:
- **TanDEM-X DEM Change Maps (DCM)** from DLR - 30m resolution pre-calculated height change values
- **RGI 7.0 Glacier Outlines** - Randolph Glacier Inventory polygons for the study region

### Methodology: DEM Differencing
The DCM products already contain the difference between two TanDEM-X acquisitions, representing surface elevation change (Δh) in meters.

---
## 1. Setup & Imports

In [ ]:
# Core libraries
import os
import zipfile
import glob
import warnings
from pathlib import Path

# Geospatial libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import box, mapping

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import TwoSlopeNorm

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print(f"Rasterio version: {rasterio.__version__}")
print(f"GeoPandas version: {gpd.__version__}")

In [ ]:
# --- CONFIGURATION ---
# Define paths relative to the project root
PROJECT_ROOT = Path.cwd()

# Data directories
DCM_ZIP_DIR = PROJECT_ROOT / "DCM"                    # Directory with zipped DCM tiles
DCM_EXTRACTED_DIR = PROJECT_ROOT / "DCM_extracted"    # Where to extract TIF files
RGI_DIR = PROJECT_ROOT / "RGI_Data"

# Input files
GLACIER_SHAPEFILE = RGI_DIR / "sikkim_bhutan_arunachal_clean.shp"

# Output directory for results
OUTPUT_DIR = PROJECT_ROOT / "results"
OUTPUT_DIR.mkdir(exist_ok=True)

# Analysis parameters
NODATA_VALUE = -32767           # TanDEM-X NoData value
MIN_VALID_CHANGE = -150         # Minimum realistic elevation change (m) - removes noise
MAX_VALID_CHANGE = 100          # Maximum realistic elevation change (m) - removes noise
PIXEL_RESOLUTION = 30           # DCM resolution in meters

# Time period (TanDEM-X baseline is ~2011-2014, change maps compare to later acquisitions)
# The "LAST1622" in filenames suggests data from 2016-2022
TIME_PERIOD_YEARS = 8           # Approximate time span for rate calculation

print(f"Project Root: {PROJECT_ROOT}")
print(f"DCM Directory: {DCM_ZIP_DIR}")
print(f"Glacier Shapefile: {GLACIER_SHAPEFILE}")

---
## 2. Data Loading

### 2.1 Extract DCM TIF Files from ZIP Archives
The DLR DCM data comes as ZIP files. We need to extract only the TIF files.

In [ ]:
def extract_dcm_tifs(zip_dir, output_dir):
    """
    Extract TIF files from TanDEM-X DCM ZIP archives.
    
    The ZIP structure is: archive.zip/FOLDER/DCM/filename.tif
    We extract only the .tif files to a flat directory.
    
    Parameters:
    -----------
    zip_dir : Path
        Directory containing the ZIP files
    output_dir : Path
        Directory to extract TIF files to
        
    Returns:
    --------
    list : Paths to extracted TIF files
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    zip_files = list(Path(zip_dir).glob("*.zip"))
    extracted_tifs = []
    
    print(f"Found {len(zip_files)} ZIP archives to process...")
    
    for i, zip_path in enumerate(zip_files, 1):
        try:
            with zipfile.ZipFile(zip_path, 'r') as zf:
                # Find TIF files in the archive (they're in the DCM subfolder)
                tif_files = [f for f in zf.namelist() 
                            if f.endswith('.tif') and '/DCM/' in f]
                
                for tif_name in tif_files:
                    # Extract just the filename
                    base_name = os.path.basename(tif_name)
                    output_path = output_dir / base_name
                    
                    # Skip if already extracted
                    if output_path.exists():
                        extracted_tifs.append(output_path)
                        continue
                    
                    # Extract the file
                    with zf.open(tif_name) as source:
                        with open(output_path, 'wb') as target:
                            target.write(source.read())
                    
                    extracted_tifs.append(output_path)
                    
            if i % 10 == 0:
                print(f"  Processed {i}/{len(zip_files)} archives...")
                
        except Exception as e:
            print(f"  Warning: Failed to process {zip_path.name}: {e}")
    
    print(f"Extraction complete! {len(extracted_tifs)} TIF files ready.")
    return extracted_tifs

In [ ]:
# Extract all TIF files
tif_files = extract_dcm_tifs(DCM_ZIP_DIR, DCM_EXTRACTED_DIR)

# Display first few files
print("\nExtracted TIF files:")
for f in sorted(tif_files)[:5]:
    print(f"  {f.name}")
print(f"  ... and {len(tif_files)-5} more")

### 2.2 Load Glacier Outlines (RGI 7.0)

In [ ]:
# Load the glacier shapefile
glaciers_gdf = gpd.read_file(GLACIER_SHAPEFILE)

# Standardize column names to lowercase for consistency
glaciers_gdf.columns = [col.lower() for col in glaciers_gdf.columns]

print(f"Loaded {len(glaciers_gdf)} glaciers from RGI shapefile")
print(f"CRS: {glaciers_gdf.crs}")
print(f"\nBounding Box:")
print(f"  Longitude: {glaciers_gdf.total_bounds[0]:.2f}° to {glaciers_gdf.total_bounds[2]:.2f}°")
print(f"  Latitude: {glaciers_gdf.total_bounds[1]:.2f}° to {glaciers_gdf.total_bounds[3]:.2f}°")
print(f"\nTotal glacier area: {glaciers_gdf['area_km2'].sum():.2f} km²")

In [ ]:
# Display available columns
print("Available attributes:")
print(glaciers_gdf.columns.tolist())

print("\nSample of glacier data:")
display_cols = ['rgi_id', 'glac_name', 'cenlon', 'cenlat', 'area_km2']
display_cols = [c for c in display_cols if c in glaciers_gdf.columns]
glaciers_gdf[display_cols].head(10)

In [ ]:
# Plot the glacier outlines to verify the study region
fig, ax = plt.subplots(figsize=(14, 8))

# Plot glaciers colored by longitude (shows spatial distribution)
glaciers_gdf.plot(
    ax=ax,
    column='cenlon',
    cmap='viridis',
    legend=True,
    legend_kwds={'label': 'Longitude (°E)', 'shrink': 0.6}
)

# Add region labels
ax.annotate('SIKKIM', xy=(88.5, 27.8), fontsize=12, fontweight='bold', color='darkred')
ax.annotate('BHUTAN', xy=(89.5, 27.5), fontsize=12, fontweight='bold', color='darkred')
ax.annotate('ARUNACHAL', xy=(92.5, 28.0), fontsize=12, fontweight='bold', color='darkred')

ax.set_title('RGI 7.0 Glacier Outlines: Sikkim, Bhutan, Arunachal Pradesh', fontsize=14)
ax.set_xlabel('Longitude (°E)')
ax.set_ylabel('Latitude (°N)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'glacier_outlines_map.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nMap saved to: {OUTPUT_DIR / 'glacier_outlines_map.png'}")

### 2.3 Inspect a Sample DCM Raster

In [ ]:
# Open one TIF to understand its structure
sample_tif = sorted(tif_files)[0]

with rasterio.open(sample_tif) as src:
    print(f"Sample Raster: {sample_tif.name}")
    print(f"="*60)
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res[0]:.6f}° x {src.res[1]:.6f}° (~{src.res[0]*111000:.1f}m)")
    print(f"Dimensions: {src.width} x {src.height} pixels")
    print(f"Bounds: {src.bounds}")
    print(f"Data type: {src.dtypes[0]}")
    print(f"NoData value: {src.nodata}")
    
    # Read a sample to check value range
    data = src.read(1)
    valid_mask = data != src.nodata
    
    print(f"\nValue Statistics (excluding NoData):")
    print(f"  Min: {data[valid_mask].min():.2f} m")
    print(f"  Max: {data[valid_mask].max():.2f} m")
    print(f"  Mean: {data[valid_mask].mean():.2f} m")
    print(f"  Valid pixels: {valid_mask.sum():,} ({100*valid_mask.sum()/data.size:.1f}%)")

---
## 3. Processing Function

This is the core function that:
1. Opens a DCM TIF file
2. Clips it to the glacier boundaries
3. Filters invalid/noisy pixels
4. Returns meaningful statistics

In [ ]:
def process_glacier_change(tif_path, glacier_gdf, 
                           nodata=NODATA_VALUE,
                           min_change=MIN_VALID_CHANGE, 
                           max_change=MAX_VALID_CHANGE):
    """
    Process a single DCM TIF file to extract glacier elevation changes.
    
    This function:
    1. Opens the raster and reprojects glaciers to match if needed
    2. Clips the raster to glacier boundaries using rasterio.mask
    3. Filters out invalid pixels (NoData and outliers)
    4. Calculates statistics for the valid glacier pixels
    
    Parameters:
    -----------
    tif_path : str or Path
        Path to the DCM GeoTIFF file
    glacier_gdf : GeoDataFrame
        Glacier outlines (will be reprojected if CRS doesn't match)
    nodata : float
        NoData value to exclude (default: -32767)
    min_change : float
        Minimum valid elevation change in meters (removes radar penetration noise)
    max_change : float
        Maximum valid elevation change in meters (removes acquisition artifacts)
    
    Returns:
    --------
    dict : Dictionary containing:
        - tile_name: Name of the processed tile
        - mean_change: Mean elevation change (m)
        - std_change: Standard deviation of change (m)
        - median_change: Median elevation change (m)
        - pixel_count: Number of valid glacier pixels
        - min_change: Minimum change value (m)
        - max_change: Maximum change value (m)
        - all_values: Array of all valid pixel values (for histograms)
    
    Returns None if the tile contains no glacier pixels.
    """
    tif_path = Path(tif_path)
    
    try:
        with rasterio.open(tif_path) as src:
            raster_crs = src.crs
            raster_bounds = src.bounds
            
            # Create a bounding box for the raster to filter glaciers
            raster_bbox = box(raster_bounds.left, raster_bounds.bottom,
                             raster_bounds.right, raster_bounds.top)
            
            # Reproject glaciers to match raster CRS if needed
            if glacier_gdf.crs != raster_crs:
                glaciers_reproj = glacier_gdf.to_crs(raster_crs)
            else:
                glaciers_reproj = glacier_gdf
            
            # Filter to only glaciers that intersect this tile
            # This significantly speeds up processing
            glaciers_in_tile = glaciers_reproj[glaciers_reproj.intersects(raster_bbox)]
            
            if len(glaciers_in_tile) == 0:
                # No glaciers in this tile - skip gracefully
                return None
            
            # Get the geometries for masking
            shapes = [mapping(geom) for geom in glaciers_in_tile.geometry]
            
            # Clip raster to glacier boundaries
            # crop=True reduces output to the extent of the shapes
            # all_touched=True includes pixels that touch the boundary
            try:
                out_image, out_transform = mask(
                    src, 
                    shapes, 
                    crop=True, 
                    all_touched=True,
                    nodata=nodata
                )
            except ValueError:
                # Shapes don't overlap with raster
                return None
            
            # Flatten the array (we only have 1 band)
            glacier_pixels = out_image[0].flatten()
            
            # === FILTERING ===
            # Step 1: Remove NoData values
            # NoData in TanDEM-X DCM is typically -32767
            valid_mask = glacier_pixels != nodata
            
            # Step 2: Remove physically unrealistic values
            # Large positive values (>100m) indicate acquisition artifacts
            # Large negative values (<-150m) can be radar penetration noise in snow/ice
            # or phase unwrapping errors
            valid_mask &= glacier_pixels >= min_change
            valid_mask &= glacier_pixels <= max_change
            
            valid_pixels = glacier_pixels[valid_mask]
            
            if len(valid_pixels) == 0:
                return None
            
            # Calculate statistics
            return {
                'tile_name': tif_path.stem,
                'glaciers_in_tile': len(glaciers_in_tile),
                'mean_change': float(np.mean(valid_pixels)),
                'std_change': float(np.std(valid_pixels)),
                'median_change': float(np.median(valid_pixels)),
                'pixel_count': int(len(valid_pixels)),
                'min_change': float(np.min(valid_pixels)),
                'max_change': float(np.max(valid_pixels)),
                'all_values': valid_pixels  # Keep for histogram
            }
            
    except Exception as e:
        print(f"  Error processing {tif_path.name}: {e}")
        return None

In [ ]:
# Test the function on one tile
print("Testing processing function on a single tile...")
test_result = process_glacier_change(sample_tif, glaciers_gdf)

if test_result:
    print(f"\nTest Results for: {test_result['tile_name']}")
    print(f"  Glaciers in tile: {test_result['glaciers_in_tile']}")
    print(f"  Valid pixels: {test_result['pixel_count']:,}")
    print(f"  Mean change: {test_result['mean_change']:.2f} m")
    print(f"  Median change: {test_result['median_change']:.2f} m")
    print(f"  Std dev: {test_result['std_change']:.2f} m")
    print(f"  Range: [{test_result['min_change']:.2f}, {test_result['max_change']:.2f}] m")
else:
    print("No glaciers found in this tile (expected for some tiles)")

---
## 4. Execution Loop

Process all DCM tiles and aggregate the results.

In [ ]:
def process_all_tiles(tif_files, glacier_gdf, verbose=True):
    """
    Process all DCM tiles and aggregate glacier change statistics.
    
    Parameters:
    -----------
    tif_files : list
        List of paths to TIF files
    glacier_gdf : GeoDataFrame
        Glacier outlines
    verbose : bool
        Print progress information
    
    Returns:
    --------
    tuple : (results_df, all_pixel_values)
        - results_df: DataFrame with per-tile statistics
        - all_pixel_values: Concatenated array of all valid pixel values
    """
    results = []
    all_values = []
    tiles_with_glaciers = 0
    
    total_tiles = len(tif_files)
    
    if verbose:
        print(f"Processing {total_tiles} DCM tiles...")
        print("="*60)
    
    for i, tif_path in enumerate(sorted(tif_files), 1):
        result = process_glacier_change(tif_path, glacier_gdf)
        
        if result is not None:
            tiles_with_glaciers += 1
            
            # Store pixel values for histogram
            all_values.append(result['all_values'])
            
            # Remove the large array from the results dict before storing
            result_clean = {k: v for k, v in result.items() if k != 'all_values'}
            results.append(result_clean)
            
            if verbose:
                print(f"  [{i:2d}/{total_tiles}] {Path(tif_path).stem}: "
                      f"{result['glaciers_in_tile']} glaciers, "
                      f"mean Δh = {result['mean_change']:.2f} m")
        else:
            if verbose and i % 10 == 0:
                print(f"  [{i:2d}/{total_tiles}] Processed (no glacier overlap)")
    
    if verbose:
        print("="*60)
        print(f"Complete! {tiles_with_glaciers}/{total_tiles} tiles contained glaciers.")
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    # Concatenate all pixel values
    all_pixel_values = np.concatenate(all_values) if all_values else np.array([])
    
    return results_df, all_pixel_values

In [ ]:
# Process all tiles
results_df, all_pixel_values = process_all_tiles(tif_files, glaciers_gdf)

print(f"\nTotal valid glacier pixels analyzed: {len(all_pixel_values):,}")

In [ ]:
# Display per-tile results
print("\nPer-Tile Results:")
print("="*80)
results_df.sort_values('mean_change')

In [ ]:
# Calculate area-weighted (pixel-count weighted) regional statistics
total_pixels = results_df['pixel_count'].sum()
pixel_area_km2 = (PIXEL_RESOLUTION ** 2) / 1e6  # Convert m² to km²

# Weighted mean (accounts for different glacier coverage in each tile)
weighted_mean_change = np.average(
    results_df['mean_change'], 
    weights=results_df['pixel_count']
)

# Direct statistics from all pixel values
overall_mean = np.mean(all_pixel_values)
overall_median = np.median(all_pixel_values)
overall_std = np.std(all_pixel_values)

print("\n" + "="*60)
print("REGIONAL SUMMARY: Sikkim, Bhutan, Arunachal Pradesh")
print("="*60)
print(f"\nTiles processed: {len(results_df)}")
print(f"Total glacier pixels: {total_pixels:,}")
print(f"Equivalent glacier area analyzed: {total_pixels * pixel_area_km2:.2f} km²")
print(f"\n--- Elevation Change Statistics ---")
print(f"Mean elevation change (Δh): {overall_mean:.2f} ± {overall_std:.2f} m")
print(f"Median elevation change: {overall_median:.2f} m")
print(f"Weighted mean (by pixel count): {weighted_mean_change:.2f} m")
print(f"\n--- Rate Estimates (over ~{TIME_PERIOD_YEARS} years) ---")
print(f"Mean thinning rate: {overall_mean/TIME_PERIOD_YEARS:.3f} m/yr")
print(f"Median thinning rate: {overall_median/TIME_PERIOD_YEARS:.3f} m/yr")

---
## 5. Visualization

### 5.1 Histogram of Elevation Changes
This histogram shows the distribution of pixel values across all glaciers. A shift toward negative values indicates overall glacier thinning.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Left plot: Full histogram ---
ax1 = axes[0]
ax1.hist(all_pixel_values, bins=100, color='steelblue', edgecolor='none', alpha=0.7)
ax1.axvline(x=0, color='black', linestyle='--', linewidth=1.5, label='Zero change')
ax1.axvline(x=overall_mean, color='red', linestyle='-', linewidth=2, 
            label=f'Mean: {overall_mean:.2f} m')
ax1.axvline(x=overall_median, color='orange', linestyle='-', linewidth=2,
            label=f'Median: {overall_median:.2f} m')

ax1.set_xlabel('Elevation Change (m)', fontsize=12)
ax1.set_ylabel('Pixel Count', fontsize=12)
ax1.set_title('Distribution of Glacier Surface Elevation Changes', fontsize=13)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Add annotation about negative shift
negative_fraction = 100 * np.sum(all_pixel_values < 0) / len(all_pixel_values)
ax1.text(0.98, 0.95, f'{negative_fraction:.1f}% of pixels\nshow thinning',
         transform=ax1.transAxes, fontsize=11, verticalalignment='top',
         horizontalalignment='right', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# --- Right plot: Zoomed view (-30 to +20 m) ---
ax2 = axes[1]
zoomed_values = all_pixel_values[(all_pixel_values >= -30) & (all_pixel_values <= 20)]
ax2.hist(zoomed_values, bins=80, color='steelblue', edgecolor='none', alpha=0.7)
ax2.axvline(x=0, color='black', linestyle='--', linewidth=1.5, label='Zero change')
ax2.axvline(x=overall_mean, color='red', linestyle='-', linewidth=2,
            label=f'Mean: {overall_mean:.2f} m')
ax2.axvline(x=overall_median, color='orange', linestyle='-', linewidth=2,
            label=f'Median: {overall_median:.2f} m')

ax2.set_xlabel('Elevation Change (m)', fontsize=12)
ax2.set_ylabel('Pixel Count', fontsize=12)
ax2.set_title('Zoomed View: -30m to +20m Range', fontsize=13)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'elevation_change_histogram.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Histogram saved to: {OUTPUT_DIR / 'elevation_change_histogram.png'}")

### 5.2 Per-Glacier Elevation Change Analysis
Compute mean elevation change for each individual glacier polygon.

In [ ]:
def compute_per_glacier_stats(tif_files, glacier_gdf,
                              nodata=NODATA_VALUE,
                              min_change=MIN_VALID_CHANGE,
                              max_change=MAX_VALID_CHANGE):
    """
    Compute elevation change statistics for each individual glacier.
    
    This function processes each glacier separately to enable per-glacier
    mapping and analysis.
    
    Returns:
    --------
    GeoDataFrame : Input glaciers with added columns for change statistics
    """
    # Initialize columns for results
    glacier_gdf = glacier_gdf.copy()
    glacier_gdf['mean_dh'] = np.nan
    glacier_gdf['median_dh'] = np.nan
    glacier_gdf['std_dh'] = np.nan
    glacier_gdf['n_pixels'] = 0
    glacier_gdf['dh_rate'] = np.nan  # m/yr
    
    print(f"Computing per-glacier statistics for {len(glacier_gdf)} glaciers...")
    print("This may take several minutes...")
    
    # Process each TIF file
    for tif_idx, tif_path in enumerate(sorted(tif_files)):
        try:
            with rasterio.open(tif_path) as src:
                raster_crs = src.crs
                raster_bounds = src.bounds
                raster_bbox = box(raster_bounds.left, raster_bounds.bottom,
                                 raster_bounds.right, raster_bounds.top)
                
                # Reproject glaciers if needed
                if glacier_gdf.crs != raster_crs:
                    glaciers_reproj = glacier_gdf.to_crs(raster_crs)
                else:
                    glaciers_reproj = glacier_gdf
                
                # Find glaciers in this tile
                intersects_mask = glaciers_reproj.intersects(raster_bbox)
                
                if not intersects_mask.any():
                    continue
                
                # Process each glacier in this tile
                for idx in glacier_gdf.index[intersects_mask]:
                    try:
                        geom = [mapping(glaciers_reproj.loc[idx, 'geometry'])]
                        
                        out_image, _ = mask(src, geom, crop=True, 
                                           all_touched=True, nodata=nodata)
                        
                        pixels = out_image[0].flatten()
                        
                        # Filter valid pixels
                        valid = (pixels != nodata) & \
                                (pixels >= min_change) & \
                                (pixels <= max_change)
                        
                        valid_pixels = pixels[valid]
                        
                        if len(valid_pixels) > 0:
                            # Accumulate statistics (handle glaciers spanning multiple tiles)
                            prev_n = glacier_gdf.loc[idx, 'n_pixels']
                            prev_mean = glacier_gdf.loc[idx, 'mean_dh']
                            
                            if prev_n > 0 and not np.isnan(prev_mean):
                                # Weighted update for glaciers spanning tiles
                                new_n = prev_n + len(valid_pixels)
                                new_mean = (prev_mean * prev_n + np.sum(valid_pixels)) / new_n
                                glacier_gdf.loc[idx, 'n_pixels'] = new_n
                                glacier_gdf.loc[idx, 'mean_dh'] = new_mean
                            else:
                                glacier_gdf.loc[idx, 'mean_dh'] = np.mean(valid_pixels)
                                glacier_gdf.loc[idx, 'median_dh'] = np.median(valid_pixels)
                                glacier_gdf.loc[idx, 'std_dh'] = np.std(valid_pixels)
                                glacier_gdf.loc[idx, 'n_pixels'] = len(valid_pixels)
                                
                    except Exception:
                        continue
                        
        except Exception as e:
            print(f"  Warning: Error processing {tif_path}: {e}")
            continue
        
        if (tif_idx + 1) % 10 == 0:
            print(f"  Processed {tif_idx + 1}/{len(tif_files)} tiles...")
    
    # Calculate thinning rate
    glacier_gdf['dh_rate'] = glacier_gdf['mean_dh'] / TIME_PERIOD_YEARS
    
    # Count glaciers with valid data
    valid_glaciers = glacier_gdf['n_pixels'] > 0
    print(f"\nComplete! {valid_glaciers.sum()}/{len(glacier_gdf)} glaciers have valid change data.")
    
    return glacier_gdf

In [ ]:
# Compute per-glacier statistics
glaciers_with_stats = compute_per_glacier_stats(tif_files, glaciers_gdf)

In [ ]:
# Filter to glaciers with valid data for visualization
valid_glaciers = glaciers_with_stats[glaciers_with_stats['n_pixels'] > 0].copy()

print(f"Glaciers with valid elevation change data: {len(valid_glaciers)}")
print(f"\nPer-glacier statistics summary:")
print(f"  Mean Δh: {valid_glaciers['mean_dh'].mean():.2f} ± {valid_glaciers['mean_dh'].std():.2f} m")
print(f"  Median Δh: {valid_glaciers['mean_dh'].median():.2f} m")
print(f"  Min Δh: {valid_glaciers['mean_dh'].min():.2f} m")
print(f"  Max Δh: {valid_glaciers['mean_dh'].max():.2f} m")

### 5.3 Map of Glacier Thinning Rates

In [ ]:
# Create the map of glacier thinning
fig, ax = plt.subplots(figsize=(16, 10))

# Use a diverging colormap centered on zero
# Blue = thickening (positive), Red = thinning (negative)
vmin = valid_glaciers['mean_dh'].quantile(0.02)  # Clip outliers
vmax = valid_glaciers['mean_dh'].quantile(0.98)

# Center the colormap at zero
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

valid_glaciers.plot(
    ax=ax,
    column='mean_dh',
    cmap='RdBu',  # Red (negative/thinning) to Blue (positive/thickening)
    norm=norm,
    legend=True,
    legend_kwds={
        'label': 'Mean Elevation Change (m)',
        'shrink': 0.6,
        'orientation': 'vertical'
    },
    edgecolor='face',
    linewidth=0.1
)

# Add region labels
ax.annotate('SIKKIM', xy=(88.5, 27.8), fontsize=14, fontweight='bold', 
            color='black', ha='center')
ax.annotate('BHUTAN', xy=(89.8, 27.3), fontsize=14, fontweight='bold',
            color='black', ha='center')
ax.annotate('ARUNACHAL\nPRADESH', xy=(93, 28.2), fontsize=14, fontweight='bold',
            color='black', ha='center')

ax.set_title('Glacier Surface Elevation Change: Sikkim, Bhutan, Arunachal Pradesh\n'
             '(TanDEM-X DEM Change Map Analysis)', fontsize=14)
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)
ax.grid(True, alpha=0.3)

# Add a text box with summary statistics
textstr = f'Glaciers analyzed: {len(valid_glaciers)}\n'
textstr += f'Mean Δh: {valid_glaciers["mean_dh"].mean():.2f} m\n'
textstr += f'Rate: {valid_glaciers["mean_dh"].mean()/TIME_PERIOD_YEARS:.3f} m/yr'

props = dict(boxstyle='round', facecolor='white', alpha=0.8)
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'glacier_thinning_map.png', dpi=200, bbox_inches='tight')
plt.show()

print(f"Map saved to: {OUTPUT_DIR / 'glacier_thinning_map.png'}")

### 5.4 Thinning Rate Distribution by Region

In [ ]:
# Classify glaciers by approximate region based on longitude
def classify_region(lon):
    """Classify glacier into region based on centroid longitude."""
    if lon < 89.0:
        return 'Sikkim'
    elif lon < 92.5:
        return 'Bhutan'
    else:
        return 'Arunachal Pradesh'

valid_glaciers['region'] = valid_glaciers['cenlon'].apply(classify_region)

# Create box plot by region
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Box plot ---
ax1 = axes[0]
regions = ['Sikkim', 'Bhutan', 'Arunachal Pradesh']
data_by_region = [valid_glaciers[valid_glaciers['region'] == r]['mean_dh'] for r in regions]

bp = ax1.boxplot(data_by_region, labels=regions, patch_artist=True)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax1.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax1.set_ylabel('Mean Elevation Change (m)', fontsize=12)
ax1.set_title('Elevation Change Distribution by Region', fontsize=13)
ax1.grid(True, alpha=0.3, axis='y')

# --- Bar plot of mean values ---
ax2 = axes[1]
region_stats = valid_glaciers.groupby('region').agg({
    'mean_dh': ['mean', 'std', 'count']
}).reset_index()
region_stats.columns = ['region', 'mean', 'std', 'count']
region_stats = region_stats.set_index('region').loc[regions].reset_index()

bars = ax2.bar(region_stats['region'], region_stats['mean'], 
               yerr=region_stats['std']/np.sqrt(region_stats['count']),  # Standard error
               capsize=5, color=colors, alpha=0.7, edgecolor='black')

ax2.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_ylabel('Mean Elevation Change (m)', fontsize=12)
ax2.set_title('Mean Elevation Change by Region (±SE)', fontsize=13)
ax2.grid(True, alpha=0.3, axis='y')

# Add count labels
for bar, count in zip(bars, region_stats['count']):
    height = bar.get_height()
    ax2.annotate(f'n={count}',
                xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, -20),
                textcoords='offset points',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'regional_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Regional comparison saved to: {OUTPUT_DIR / 'regional_comparison.png'}")

---
## 6. Results & Interpretation

In [ ]:
# Create summary table by region
print("="*80)
print("SUMMARY TABLE: Glacier Surface Elevation Change")
print("Region: Sikkim, Bhutan, Arunachal Pradesh (Eastern Himalaya)")
print("Data: TanDEM-X DEM Change Map (DCM)")
print(f"Time Period: ~{TIME_PERIOD_YEARS} years (TanDEM-X baseline to recent)")
print("="*80)

summary_data = []

for region in ['Sikkim', 'Bhutan', 'Arunachal Pradesh']:
    region_data = valid_glaciers[valid_glaciers['region'] == region]
    
    if len(region_data) > 0:
        summary_data.append({
            'Region': region,
            'Glaciers': len(region_data),
            'Total Area (km²)': region_data['area_km2'].sum(),
            'Mean Δh (m)': region_data['mean_dh'].mean(),
            'Std Δh (m)': region_data['mean_dh'].std(),
            'Median Δh (m)': region_data['mean_dh'].median(),
            'Rate (m/yr)': region_data['mean_dh'].mean() / TIME_PERIOD_YEARS
        })

# Add total row
summary_data.append({
    'Region': 'TOTAL/OVERALL',
    'Glaciers': len(valid_glaciers),
    'Total Area (km²)': valid_glaciers['area_km2'].sum(),
    'Mean Δh (m)': valid_glaciers['mean_dh'].mean(),
    'Std Δh (m)': valid_glaciers['mean_dh'].std(),
    'Median Δh (m)': valid_glaciers['mean_dh'].median(),
    'Rate (m/yr)': valid_glaciers['mean_dh'].mean() / TIME_PERIOD_YEARS
})

summary_df = pd.DataFrame(summary_data)

# Format the table
print("\n")
print(summary_df.to_string(index=False, 
                           formatters={
                               'Total Area (km²)': '{:.1f}'.format,
                               'Mean Δh (m)': '{:.2f}'.format,
                               'Std Δh (m)': '{:.2f}'.format,
                               'Median Δh (m)': '{:.2f}'.format,
                               'Rate (m/yr)': '{:.3f}'.format
                           }))

print("\n" + "="*80)

In [ ]:
# Estimate total volume change
pixel_area_m2 = PIXEL_RESOLUTION ** 2  # 900 m²

# Using the pixel-level data for more accurate volume estimation
total_volume_change_m3 = np.sum(all_pixel_values) * pixel_area_m2
total_volume_change_km3 = total_volume_change_m3 / 1e9

# Equivalent water loss (ice density ~900 kg/m³)
ice_density = 900  # kg/m³
water_density = 1000  # kg/m³
water_equivalent_km3 = total_volume_change_km3 * (ice_density / water_density)

print("\n" + "="*60)
print("VOLUME CHANGE ESTIMATES")
print("="*60)
print(f"\nTotal glacier pixels analyzed: {len(all_pixel_values):,}")
print(f"Total area analyzed: {len(all_pixel_values) * pixel_area_m2 / 1e6:.2f} km²")
print(f"\nTotal volume change: {total_volume_change_km3:.3f} km³")
print(f"Water equivalent: {water_equivalent_km3:.3f} km³")
print(f"\nAnnual rate: {total_volume_change_km3/TIME_PERIOD_YEARS:.4f} km³/yr")
print(f"Annual water equivalent: {water_equivalent_km3/TIME_PERIOD_YEARS:.4f} km³/yr")

print("\n" + "-"*60)
print("INTERPRETATION NOTES:")
print("-"*60)
print("""
• Negative values indicate surface lowering (glacier thinning/mass loss)
• The TanDEM-X DCM captures surface elevation change, not direct mass change
• Radar penetration into snow/firn can affect measurements in accumulation zones
• Values filtered to remove obvious noise (> +100m or < -150m)
• Time period is approximate based on TanDEM-X acquisition dates
• For accurate mass balance, density assumptions are required
""")

In [ ]:
# Save results to CSV
summary_df.to_csv(OUTPUT_DIR / 'regional_summary.csv', index=False)
print(f"Summary table saved to: {OUTPUT_DIR / 'regional_summary.csv'}")

# Save per-glacier results
export_cols = ['rgi_id', 'glac_name', 'cenlon', 'cenlat', 'area_km2', 
               'mean_dh', 'median_dh', 'std_dh', 'n_pixels', 'dh_rate', 'region']
export_cols = [c for c in export_cols if c in valid_glaciers.columns]

valid_glaciers[export_cols].to_csv(OUTPUT_DIR / 'per_glacier_results.csv', index=False)
print(f"Per-glacier results saved to: {OUTPUT_DIR / 'per_glacier_results.csv'}")

# Save the GeoDataFrame with results as a new shapefile
valid_glaciers.to_file(OUTPUT_DIR / 'glaciers_with_change.shp')
print(f"Shapefile with results saved to: {OUTPUT_DIR / 'glaciers_with_change.shp'}")

---
## 7. Additional Analysis: Elevation Dependence

In [ ]:
# Check if elevation data is available in the RGI shapefile
elev_cols = [c for c in valid_glaciers.columns if 'elev' in c.lower() or 'zmed' in c.lower()]
print(f"Available elevation columns: {elev_cols}")

if 'zmed' in valid_glaciers.columns or 'zmed_m' in valid_glaciers.columns:
    elev_col = 'zmed' if 'zmed' in valid_glaciers.columns else 'zmed_m'
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scatter plot of elevation change vs median elevation
    scatter = ax.scatter(
        valid_glaciers[elev_col],
        valid_glaciers['mean_dh'],
        c=valid_glaciers['area_km2'],
        cmap='viridis',
        alpha=0.6,
        s=20
    )
    
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_xlabel('Median Glacier Elevation (m a.s.l.)', fontsize=12)
    ax.set_ylabel('Mean Elevation Change (m)', fontsize=12)
    ax.set_title('Elevation Change vs. Glacier Altitude', fontsize=13)
    ax.grid(True, alpha=0.3)
    
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.8)
    cbar.set_label('Glacier Area (km²)', fontsize=11)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'elevation_dependence.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Elevation dependence plot saved to: {OUTPUT_DIR / 'elevation_dependence.png'}")
else:
    print("Median elevation data not available in shapefile.")
    print("Skipping elevation dependence analysis.")

---
## Summary

This analysis processed TanDEM-X DEM Change Maps (DCM) to estimate glacier surface elevation changes in the Sikkim, Bhutan, and Arunachal Pradesh regions of the Eastern Himalaya.

### Key Outputs:
1. **Regional Statistics**: Mean, median, and standard deviation of elevation change by region
2. **Per-Glacier Data**: Individual glacier statistics saved as CSV and Shapefile
3. **Visualizations**: 
   - Histogram of pixel-level elevation changes
   - Map of per-glacier thinning rates
   - Regional comparison plots

### Limitations:
- Time period is approximate (based on TanDEM-X naming conventions)
- Radar penetration effects not corrected
- Void pixels and data gaps may affect some glaciers
- Volume-to-mass conversion requires density assumptions

### Next Steps:
- Compare with independent datasets (ICESat-2, ASTER)
- Apply glacier-specific penetration corrections
- Analyze temporal trends if multi-epoch data available